# Preprocess test and train dataset

In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Wed Jun  3 18:25:19 2020

@author: orestis
"""

from pandas import DataFrame,read_csv

import pandas as pd
import nltk
import string

#Location of train data
TrainLocation = '/home/orestis/Desktop/GitHubRepositories/DataMining-MachineLearning/CommentsClassification/data/train.csv'
#Location of test data
TestLocation = '/home/orestis/Desktop/GitHubRepositories/DataMining-MachineLearning/CommentsClassification/data/test.csv'

train_df = pd.read_csv(TrainLocation,names=['Class','Date','Comment'],skiprows=1)   #Get the training data
test_df = pd.read_csv(TestLocation,names=['Class','Date','Comment'],skiprows=1)     #Get the testing data

#Delete date column
del train_df['Date']
del test_df['Date']

train_comments = train_df['Comment']
test_comments = test_df['Comment']

#PREPROCESS COMMENTS DATA
alphabet = list(string.ascii_lowercase)
numbers=['1','2','3','4','5','6','7','8','9','0',' ']
alphabet = alphabet + numbers
#Convert to lower case
for i in range(len(train_comments)):
    train_comments[i]=train_comments[i].lower()
    train_comments[i] = train_comments[i].replace('-',' ')
    train_comments[i] = train_comments[i].replace('\\',' ')
    train_comments[i] = train_comments[i].replace('.',' ')
    train_comments[i] = train_comments[i].replace(',',' ')
    train_comments[i] = train_comments[i].replace('_',' ')
    #Keep only the letters and numbers
    for c in train_comments[i]:
        if c not in alphabet:
             train_comments[i]=train_comments[i].replace(c,'')

for i in range(len(test_comments)):
    test_comments[i]=test_comments[i].lower()
    test_comments[i] = test_comments[i].replace('-',' ')
    test_comments[i] = test_comments[i].replace('\\',' ')
    test_comments[i] = test_comments[i].replace('.',' ')
    test_comments[i] = test_comments[i].replace(',',' ')
    test_comments[i] = test_comments[i].replace('_',' ')
    #Keep only the letters and numbers
    for c in test_comments[i]:
        if c not in alphabet:
             test_comments[i]=test_comments[i].replace(c,'')
    
train_class_values = train_df['Class']

#Create the new data sets
NewTrainDataSet = list(zip(train_comments,train_class_values))
NewTestDataSet = list(test_comments)

new_train_df = pd.DataFrame(data=NewTrainDataSet,columns=['Comments','Classs'])
new_test_df = pd.DataFrame(data=NewTestDataSet,columns=['Comments'])

new_train_df.to_csv('train.csv',index=False,header=False)
new_test_df.to_csv('test.csv',index=False,header=False)

# CountVectorizer and Naive Bayes Prediction

In [7]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  4 09:29:45 2020

@author: orestis
"""

from pandas import DataFrame,read_csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB

import pandas as pd

#Function to get accuracy of the predicted results
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

TrainDataLocation = './train.csv'
TestDataLocation = './test.csv'

train_df = pd.read_csv(TrainDataLocation,header=None,names=['Comment','Class'])
test_df = pd.read_csv(TestDataLocation,header=None,names=['Comment','Class'])

train_comments = train_df['Comment']
test_comments = test_df['Comment']

train_classes=train_df['Class']
test_classes =test_df['Class']


#Create a Gaussian Naive Bayes model
model = GaussianNB()

#Create Train Count Vectorizer
TrainCount_Vectorizer = list()

vectorizer = CountVectorizer()
predicted_classes=list()
#Make the predictions
for test_comment in test_comments:
    TrainCount_Vectorizer=[]
    #tokenize and build vocabulary
    vectorizer.fit([test_comment])
    for x in train_comments:
        comment = [x]
        #encode the document
        vector = vectorizer.transform(comment)
        temp_array = vector.toarray()
        TrainCount_Vectorizer.append(temp_array[0])
    
    #Predict Output
    model.fit(TrainCount_Vectorizer,train_classes)
    vector = vectorizer.transform([test_comment])
    predicted = model.predict(vector.toarray())
    predicted_classes.append(predicted)

score = accuracy_metric(test_classes, predicted_classes)
print('Score:',score)

/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in

Score: 60.581655480984345


# Prediction After removing stopwords and lemmization

In [11]:

# -*- coding: utf-8 -*-
"""
Created on Thu Jun  4 11:29:23 2020

@author: orestis
"""

from pandas import DataFrame,read_csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import pandas as pd

#Function to get accuracy of the predicted results
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(predicted)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

TrainDataLocation = './train.csv'
TestDataLocation = './test.csv'

train_df = pd.read_csv(TrainDataLocation,header=None,names=['Comment','Class'])
test_df = pd.read_csv(TestDataLocation,header=None,names=['Comment','Class'])

train_comments = train_df['Comment']
test_comments = test_df['Comment']

train_classes=train_df['Class']
test_classes=test_df['Class']

lemmatizer = WordNetLemmatizer()
#Create a set of stopwords
stop_words = set(stopwords.words('english'))

#Create a new comment train dataset
#After lemmazation and removing stopwords
cleaned_train_comments = list()
for comment in train_comments:
    example_sent = comment
    word_tokens = word_tokenize(example_sent)
    filtered_sentence = [] 
      
    for w in word_tokens: 
        if w not in stop_words: 
            lemmatizer.lemmatize(w)
            filtered_sentence.append(w) 
    
    new_sentence=''
    for word in filtered_sentence:
        new_sentence = new_sentence + word + ' '
    
    cleaned_train_comments.append(new_sentence)

#Do the same for the test comments dataset
cleaned_test_comments=list()
for comment in test_comments:
    example_sent = comment
    word_tokens = word_tokenize(example_sent)
    filtered_sentence = [] 
      
    for w in word_tokens: 
        if w not in stop_words: 
            lemmatizer.lemmatize(w)
            filtered_sentence.append(w) 
    
    new_sentence=''
    for word in filtered_sentence:
        new_sentence = new_sentence + word + ' '
    
    cleaned_test_comments.append(new_sentence)


#Create a Gaussian Naive Bayes model
model = GaussianNB()

#Create Train Count Vectorizer
TrainCount_Vectorizer = list()

vectorizer = CountVectorizer()
predicted_classes=list()
#Make the predictions
for test_comment in cleaned_test_comments:
    TrainCount_Vectorizer=[]
    #tokenize and build vocabulary
    try:
        vectorizer.fit([test_comment])
    except:
         TrainCount_Vectorizer.append(temp_array[0])
         continue   
    for x in cleaned_train_comments:
        comment = [x]
        #encode the document
        vector = vectorizer.transform(comment)
        temp_array = vector.toarray()
        TrainCount_Vectorizer.append(temp_array[0])
    
    #Predict Output
    model.fit(TrainCount_Vectorizer,train_classes)
    vector = vectorizer.transform([test_comment])
    predicted = model.predict(vector.toarray())
    predicted_classes.append(predicted)

score = accuracy_metric(test_classes, predicted_classes)
print('Score:',score)

/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in

/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in subtract
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: divide by zero encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: divide by zero encountered in true_divide
  (self.sigma_[i, :]), 1)
/home/orestis/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:452: RuntimeWarning: invalid value encountered in

Score: 52.66219239373602
